## Réinjection des données de liage dans le fichier XML

In [ ]:
# Utilitaire de conversion
'''
import pandas as pd

# Chemin vers le fichier ODS
chemin_fichier_ods = '../../utils/pouille/out/linking_out/liage_po7.ods'

# Lire le fichier ODS en utilisant pandas
dataframe = pd.read_excel(chemin_fichier_ods, engine='odf')

# Chemin vers le fichier CSV de sortie
chemin_fichier_csv = '../../utils/pouille/out/linking_out/liage_po7.csv'

# Enregistrer le dataframe en tant que fichier CSV en conservant les noms des colonnes
dataframe.to_csv(chemin_fichier_csv, index=False)
'''

In [ ]:
import pandas as pd
from lxml import etree as ET

# Charger le fichier XML
tree = ET.parse('../../data/PO_t7/PO_t7.xml')
root = tree.getroot()

# Charger le DataFrame
df = pd.read_csv('../../utils/pouille/out/linking_out/liage_po7.csv', dtype=str)

# Pour chaque article dans le XML
for article in root.findall('article'):
    # Récupérer l'article_id
    article_id = article.get('old-id')

    # Vérifier si l'article_id est présent dans le DataFrame
    if article_id in df['article_id'].values:
        # Récupérer la ligne correspondante dans le DataFrame
        df_row = df.loc[df['article_id'] == article_id]

        # Vérifier si localisationco est "none" et insee_code est présent
        if df_row['localisationco'].values[0] == 'none' and pd.notnull(df_row['insee_code'].values[0]):
            # Ajouter l'attribut type="commune" à la balise article
            article.set('type', 'commune')

            # Ajouter la balise <insee> à la balise article avec le code INSEE
            insee_code = df_row['insee_code'].values[0]
            insee_elem = ET.Element('insee')
            insee_elem.text = insee_code
            article.append(insee_elem)

        # Vérifier si localisationco est présent et insee_code est présent
        if pd.notnull(df_row['localisationco'].values[0]) and pd.notnull(df_row['insee_code'].values[0]):
            loc_elem = article.find('localisationco')
            if loc_elem is not None:
                search_str = ET.tostring(loc_elem, encoding='unicode')
                insee_code = df.loc[df['article_id'] == article_id, 'insee_code'].values[0]
                corresponding_row = df.loc[df['article_id'] == article_id]
                localisationco = corresponding_row['localisationco'].values[0]
                replaced_str = search_str.replace(localisationco, f'<commune insee="{insee_code}">{localisationco}</commune>')
                new_loc_elem = ET.fromstring(replaced_str)
                article.replace(loc_elem, new_loc_elem)

# Enregistrer le fichier XML modifié
tree.write('../../utils/pouille/out/linking_out/PO_t7_modified.xml', encoding='utf-8', xml_declaration=True)
